<a href="https://colab.research.google.com/github/Abhijit036/NLP/blob/main/Word_Embedding_%26_Sentiment_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 BERT for Word Embeddings


This notebook provides a **very simplified and beginner-friendly** guide to using **BERT for sentiment analysis**. We'll:
- Use a small set of sentences labeled as Positive (1) or Negative (0)
- Tokenize them using BERT tokenizer
- Get BERT embeddings
- (Optionally) Use these embeddings for a classifier

All steps are done using `transformers` and `torch` libraries.


In [1]:
!pip install transformers torch --quiet

In [2]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd

### 🧾 Step 1: Create Sample Data

In [3]:

data = {
    'text': [
        "I love this product!",
        "This is the worst experience I've ever had.",
        "Absolutely fantastic!",
        "Not good, very disappointing.",
        "I'm happy with the service."
    ],
    'label': [1, 0, 1, 0, 1]  # 1 = Positive, 0 = Negative
}

df = pd.DataFrame(data)
df

,text,label
0,I love this product!,1
1,This is the worst experience I've ever had.,0
2,Absolutely fantastic!,1
3,"Not good, very disappointing.",0
4,I'm happy with the service.,1


### 🧠 Step 2: Tokenize Text using BERT Tokenizer

* Loads a pre-trained BERT tokenizer.
* ``'bert-base-uncased'`` means:
    * Base BERT model with 12 layers
    * Trained on lowercase text (uncased: "Hello" = "hello")


Tokenizer is responsible for splitting the sentence into Word Piece tokens.

This tokenizes a **list of text** from your DataFrame using the tokenizer.

* **list(df['text']):** Extracts a list of raw strings from the DataFrame's 'text' column.

* **padding=True:** Ensures that all sequences are the same length by padding shorter ones with [PAD] tokens. After the actual tokens.

* **truncation=True:** Cuts off longer sentences so they don’t exceed the max length BERT supports (maximum sequence length is 512 tokens.).

* **return_tensors="pt":** Converts output into PyTorch tensors instead of Python lists or NumPy arrays.

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize with padding and truncation
inputs = tokenizer(
    list(df['text']),
    padding=True,
    truncation=True, #any sentence is longer than BERT’s maximum allowed length (512 tokens), it will be cut off
    return_tensors="pt"
)

inputs['input_ids'].shape  # (batch_size, max_seq_len)

# Splits text into word pieces (subwords) → e.g., "playing" → ["play", "##ing"].

# Maps tokens to integer IDs using BERT’s vocabulary (30,522 tokens for bert-base).

# Adds special tokens like [CLS] (classification) at the start, [SEP] (separator) at the end.

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

torch.Size([5, 13])

In [5]:
# Sentence 1: "I love this product!"
# Sentence 2: "This is bad."
# Sentence 3: "Absolutely fantastic!"

| Sentence | Tokens (IDs)                        |                          |
| -------- | ----------------------------------- | ------------------------ |
| S1       | \[101, 1045, 2293, 2023, 4033, 102] |                          |
| S2       | \[101, 2023, 2003, 2919, 102, 0]    | <- padded to same length |
| S3       | \[101, 6934, 1033, 102, 0, 0]       | <- padded                |

101 = [CLS], 102 = [SEP], 0 = [PAD]

SyntaxError: invalid syntax (ipython-input-3982321406.py, line 5)

# Sentence 1: "I love this product!"
# Sentence 2: "This is bad."
# Sentence 3: "Absolutely fantastic!"

| Sentence | Tokens (IDs)                        |                          |
| -------- | ----------------------------------- | ------------------------ |
| S1       | \[101, 1045, 2293, 2023, 4033, 102] |                          |
| S2       | \[101, 2023, 2003, 2919, 102, 0]    | <- padded to same length |
| S3       | \[101, 6934, 1033, 102, 0, 0]       | <- padded                |

101 = [CLS], 102 = [SEP], 0 = [PAD]

In [6]:
input = "I Love AI!"

tokenizer.tokenize(input)

['i', 'love', 'ai', '!']

### 📥 Step 3: Get Sentence Embeddings using BERT

In [7]:
model = BertModel.from_pretrained("bert-base-uncased")
with torch.no_grad():
    outputs = model(**inputs) #This tensor contains contextual embeddings for each token in each sentence.
# inputs = {
#     'input_ids': tensor(...),      # token IDs of your sentences
#     'attention_mask': tensor(...), # 1 for real tokens, 0 for padding
#     # 'token_type_ids' may also be here for BERT (for sentence pairs)
# }
# **inputs unpacks the dictionary into keyword arguments.

# So this is equivalent to writing:

# outputs = model(
#     input_ids=inputs['input_ids'],
#     attention_mask=inputs['attention_mask'],
#     token_type_ids=inputs.get('token_type_ids', None)
# )

# outputs.last_hidden_state shape: (batch, seq_len, hidden_size)
# We'll use the embedding of [CLS] token for sentence representation
sentence_embeddings = outputs.last_hidden_state[:, 0, :]  # shape: (batch, token_sequence, hidden_size/vector)
#: → all batches → pick all sentences (dimension 0)

#0 → first token in the sequence → [CLS] token (dimension 1)

#: → all hidden dimensions → 0 to 767 (dimension 2)

# So we are trying to cover all the contextual meaning of the sentences, covering classification as well

sentence_embeddings.shape



model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

torch.Size([5, 768])

In [8]:
# Sentence 1: "I love this product!"
# Sentence 2: "This is bad."
# Sentence 3: "Absolutely fantastic!"

| Sentence | Tokens (IDs)                        |                          |
| -------- | ----------------------------------- | ------------------------ |
| S1       | \[101, 1045, 2293, 2023, 4033, 102] |                          |
| S2       | \[101, 2023, 2003, 2919, 102, 0]    | <- padded to same length |
| S3       | \[101, 6934, 1033, 102, 0, 0]       | <- padded                |

101 = [CLS], 102 = [SEP], 0 = [PAD]

SyntaxError: invalid syntax (ipython-input-3982321406.py, line 5)

### 🔍 Step 4: View Embeddings

In [ ]:
sentence_embeddings  # These are 768-dimensional vectors for each sentence

### ✅ Summary


- We used `BertTokenizer` to tokenize text
- Passed input to `BertModel` to get embeddings
- Used [CLS] token's embedding as a sentence representation

This can now be used as input features to a classifier (e.g., logistic regression, neural net).


In [9]:
# Add the Embeddings Back to DataFrame

df['embedding'] = sentence_embeddings.tolist()
df

,text,label,embedding
0,I love this product!,1,"[0.1876906454563141, 0.2440350502729416, 0.101..."
1,This is the worst experience I've ever had.,0,"[-0.15914830565452576, 0.4611494541168213, -0...."
2,Absolutely fantastic!,1,"[-0.05521172285079956, 0.1704026609659195, 0.0..."
3,"Not good, very disappointing.",0,"[-0.43618062138557434, 0.015502016060054302, -..."
4,I'm happy with the service.,1,"[0.1945129781961441, 0.2415696382522583, -0.04..."


In [10]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Convert embeddings and labels to numpy arrays
X = sentence_embeddings.numpy()
y = df['label'].values

# Split data into training and testing sets (optional for this small dataset, but good practice)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Logistic Regression model
log_reg = LogisticRegression(solver='liblinear') # 'liblinear' is a good choice for small datasets
log_reg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = log_reg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy:.2f}")

# You can also predict on the training set to see how well it fits the training data
y_train_pred = log_reg.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Accuracy on the training set: {train_accuracy:.2f}")

# To predict the sentiment of a new sentence:
# 1. Tokenize the new sentence
# 2. Get its BERT embedding
# 3. Use the trained Logistic Regression model to predict the label

def predict_sentiment(text, model, tokenizer, classifier):
    inputs = tokenizer(
        [text],
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    with torch.no_grad():
        outputs = model(**inputs)
    sentence_embedding = outputs.last_hidden_state[:, 0, :].numpy()
    prediction = classifier.predict(sentence_embedding)
    return "Positive" if prediction[0] == 1 else "Negative"

# Example prediction on a new sentence
new_sentence = "This is an amazing day!"
predicted_sentiment = predict_sentiment(new_sentence, model, tokenizer, log_reg)
print(f"The sentiment of '{new_sentence}' is: {predicted_sentiment}")

new_sentence_2 = "I hate this situation."
predicted_sentiment_2 = predict_sentiment(new_sentence_2, model, tokenizer, log_reg)
print(f"The sentiment of '{new_sentence_2}' is: {predicted_sentiment_2}")


Accuracy on the test set: 0.00
Accuracy on the training set: 1.00
The sentiment of 'This is an amazing day!' is: Positive
The sentiment of 'I hate this situation.' is: Positive
